In [1]:
import pandas as pd
import numpy as np

**Load the data**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd '/content/drive/My Drive'
netflix = pd.read_csv('./proyecto Netflix/netflix_titles.csv')
netflix.head()

/content/drive/My Drive


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


**dimension de nuestro df**

In [8]:
netflix.shape

(6234, 12)

In [9]:
netflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6234 entries, 0 to 6233
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       6234 non-null   int64 
 1   type          6234 non-null   object
 2   title         6234 non-null   object
 3   director      4265 non-null   object
 4   cast          5664 non-null   object
 5   country       5758 non-null   object
 6   date_added    6223 non-null   object
 7   release_year  6234 non-null   int64 
 8   rating        6224 non-null   object
 9   duration      6234 non-null   object
 10  listed_in     6234 non-null   object
 11  description   6234 non-null   object
dtypes: int64(2), object(10)
memory usage: 584.6+ KB


**revisamos cantidad de datos nulos por variable**

In [10]:
netflix.notnull().apply(pd.Series.value_counts)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
False,NaN,NaN,NaN,1969,570,476,11,NaN,10,NaN,NaN,NaN
True,6234.0,6234.0,6234.0,4265,5664,5758,6223,6234.0,6224,6234.0,6234.0,6234.0


In [11]:
df = netflix[['title','director','cast','listed_in','description']]
df.head(5)

,title,director,cast,listed_in,description
0,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,Jandino: Whatever it Takes,NaN,Jandino Asporaat,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",Kids' TV,"With the help of three human allies, the Autob..."
3,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",Kids' TV,When a prison ship crash unleashes hundreds of...
4,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",Comedies,When nerdy high schooler Dani finally attracts...


**Transform the data**

In [12]:
!pip install rake-nltk

from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=7819 sha256=81bd12e9799b88ed795011e60df122377291dbb589894e964bcc2e73283ad8b2
  Stored in directory: /root/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk


**eliminamos valores nulos**



In [13]:
df.dropna(inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
df.shape

(3909, 5)

In [15]:
df['key_words'] = ''

for index,row in df.iterrows():
    description = row['description']
    
    #instanciamos a Rake para separar palabras comunes y caracteres de puntuacion
    rake = Rake()

    #extraemos las palabras de la descripcion
    rake.extract_keywords_from_text(description)

    #obtenemos un diccionario con las palabras claves como llaves y el score como sus valores
    key_words_dict_scores = rake.get_word_degrees()

    #asignamos las palabras claves a la nueva columna a su correspondiente fila
    row['key_words'] = list(key_words_dict_scores.keys())


#elimino la columna descripcion
df.drop(columns=['description'], inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [17]:
df.head(2)

,title,director,cast,listed_in,key_words
0,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","Children & Family Movies, Comedies","[awesome, wedding, polar, bear, king, must, ta..."
4,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",Comedies,"[interest, ex, longtime, crush, cross, hairs, ..."


In [18]:
#vamos a convertir la columna director en una lista y separamos por coma donde habia espacio
df['director'] = df['director'].map(lambda x: x.split(' '))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
#vamos a convertir la columna de actores en una lista y separar por comas donde habia coma
df['cast'] = df['cast'].map(lambda x: x.split(',')[:3])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [25]:
#vamos a convertir en lista el tipo de pelicula y separar por comas donde gabia coma
df['listed_in'] = df['listed_in'].map(lambda x: x.lower().split(','))

AttributeError: ignored

In [26]:
#vamos a unir nombres y apellidos de directores y actores para que sean solo uno
for _, row in df.iterrows():
    row['cast'] = [x.lower().replace(' ', '') for x in row['cast']]
    row['director'] = ''.join(row['director']).lower()

In [27]:
df.set_index('title', inplace=True)
df.head(2)

,director,cast,listed_in,key_words
title,,,,
Norm of the North: King Sized Adventure,"richardfinn,timmaltby","[alanmarriott, andrewtoth, briandobson]","[children & family movies, comedies]","[awesome, wedding, polar, bear, king, must, ta..."
#realityhigh,fernandolebrija,"[nestacooper, katewalsh, johnmichaelhiggins]",[comedies],"[interest, ex, longtime, crush, cross, hairs, ..."


In [28]:
#vamos a unir todas las features en una sola canasta de palabaras para poder encontrar las similitud
df['bag_of_words'] = ''
columns = df.columns

for _, row in df.iterrows():
    words = ''
    for col in columns:
        if col != 'director':
            words = words + ' '.join(row[col]) + ' '
        else:
            words = words + row[col] + ' '
    row['bag_of_words'] = words

df.drop(columns= [col for col in df.columns if col != 'bag_of_words'], inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [29]:
df.head(5)

,bag_of_words
title,
Norm of the North: King Sized Adventure,"richardfinn,timmaltby alanmarriott andrewtoth ..."
#realityhigh,fernandolebrija nestacooper katewalsh johnmich...
Automata,gabeibáñez antoniobanderas dylanmcdermott mela...
Fabrizio Copano: Solo pienso en mi,"rodrigotoro,franciscoschultz fabriziocopano st..."
Good People,henrikrubengenz jamesfranco katehudson tomwilk...


**Modeling**

In [30]:
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

#geenrando la matriz de coseno de similaridad
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [32]:
cosine_sim

array([[1.        , 0.04454354, 0.04166667, ..., 0.        , 0.        ,
        0.03450328],
       [0.04454354, 1.        , 0.        , ..., 0.        , 0.03253   ,
        0.03688556],
       [0.04166667, 0.        , 1.        , ..., 0.03311331, 0.03042903,
        0.03450328],
       ...,
       [0.        , 0.        , 0.03311331, ..., 1.        , 0.31437304,
        0.38388595],
       [0.        , 0.03253   , 0.03042903, ..., 0.31437304, 1.        ,
        0.32756921],
       [0.03450328, 0.03688556, 0.03450328, ..., 0.38388595, 0.32756921,
        1.        ]])

In [33]:
#creamos una serie de indices para hacer match por ellos
indices = pd.Series(df.index)
indices[:2]

0    Norm of the North: King Sized Adventure
1                               #realityhigh
Name: title, dtype: object

In [34]:
#definiendo funcion para tomar los titulos como input y retornar top15 reomendadas
def recommendations(title, cosine_sim=cosine_sim):
    recommended_movies = []

    idx = indices[indices==title].index[0]

    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending=False)

    top15_indexes = list(score_series.iloc[1:16].index)

    for i in top15_indexes:
        recommended_movies.append(list(df.index)[i])

    return recommended_movies

In [36]:
recommendations('The Bleeder')

['Shabd',
 'The Brawler',
 'Rocky IV',
 'Rocky III',
 'Masoom',
 'Mercenary',
 'A Perfect Man',
 '100 Meters',
 'Kuppivala',
 'Time Share',
 'Mary Kom',
 'Seven and a half dates',
 'The Danish Girl',
 'Rocky II',
 'Prague']

In [37]:
recommendations('Breaking Bad')

IndexError: ignored

#df sin la feature director

In [38]:
df_2 = netflix[['title','cast','listed_in','description']]
df_2.head(2)

,title,cast,listed_in,description
0,Norm of the North: King Sized Adventure,"Alan Marriott, Andrew Toth, Brian Dobson, Cole...","Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,Jandino: Whatever it Takes,Jandino Asporaat,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...


In [39]:
df_2[df_2['title'] == 'Stranger Things']

,title,cast,listed_in,description
5931,Stranger Things,"Winona Ryder, David Harbour, Finn Wolfhard, Mi...","TV Horror, TV Mysteries, TV Sci-Fi & Fantasy","When a young boy vanishes, a small town uncove..."


In [40]:
df_2[df_2['title'] == 'Breaking Bad']

,title,cast,listed_in,description
6160,Breaking Bad,"Bryan Cranston, Aaron Paul, Anna Gunn, Dean No...","Crime TV Shows, TV Dramas, TV Thrillers",A high school chemistry teacher dying of cance...


In [41]:
df_2.dropna(inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [42]:
df_2['key_words'] = ''

for index,row in df_2.iterrows():
    description = row['description']
    
    #instanciamos a Rake para separar palabras comunes y caracteres de puntuacion
    r = Rake()

    #extraemos las palabras de la descripcion
    r.extract_keywords_from_text(description)

    #obtenemos un diccionario con las palabras claves como llaves y el score como sus valores
    key_words_dict_scores_2 = r.get_word_degrees()

    #asignamos las palabras claves a la nueva columna a su correspondiente fila
    row['key_words'] = list(key_words_dict_scores_2.keys())


#elimino la columna descripcion
df_2.drop(columns=['description'], inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [43]:
df_2['cast'] = df_2['cast'].map(lambda x: x.split(','))

df_2['listed_in'] = df_2['listed_in'].map(lambda x: x.lower().split(','))

for _,row in df_2.iterrows():
    row['cast'] = [x.lower().replace(' ', '') for x in row['cast']]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [44]:
df_2.set_index('title', inplace=True)
df_2.head(2)

,cast,listed_in,key_words
title,,,
Norm of the North: King Sized Adventure,"[alanmarriott, andrewtoth, briandobson, coleho...","[children & family movies, comedies]","[awesome, wedding, polar, bear, king, must, ta..."
Jandino: Whatever it Takes,[jandinoasporaat],[stand-up comedy],"[comedy, show, audience, fire, serenades, rais..."


In [45]:
df_2.shape

(5664, 3)

In [46]:
df_2['bag_of_words'] = ''
columns_2 = df_2.columns

for _, row in df_2.iterrows():
    words_2 = ''
    for col in columns_2:
        words_2 = words_2 + ' '.join(row[col]) + ' '
    row['bag_of_words'] = words_2

df_2.drop(columns= [col for col in df_2.columns if col != 'bag_of_words'], inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [47]:
df_2.head()

,bag_of_words
title,
Norm of the North: King Sized Adventure,alanmarriott andrewtoth briandobson colehoward...
Jandino: Whatever it Takes,jandinoasporaat stand-up comedy comedy show au...
Transformers Prime,petercullen sumaleemontano frankwelker jeffrey...
Transformers: Robots in Disguise,willfriedle darrencriss constancezimmer kharyp...
#realityhigh,nestacooper katewalsh johnmichaelhiggins keith...


In [48]:
count2 = CountVectorizer()
count_matrix2 = count2.fit_transform(df_2['bag_of_words'])

#geenrando la matriz de coseno de similaridad
cosine_sim2 = cosine_similarity(count_matrix2, count_matrix2)

In [49]:
cosine_sim2

array([[1.        , 0.        , 0.        , ..., 0.04376881, 0.03138824,
        0.03448276],
       [0.        , 1.        , 0.04472136, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.04472136, 1.        , ..., 0.04714045, 0.10141851,
        0.07427814],
       ...,
       [0.04376881, 0.        , 0.04714045, ..., 1.        , 0.15936381,
        0.13130643],
       [0.03138824, 0.        , 0.10141851, ..., 0.15936381, 1.        ,
        0.21971769],
       [0.03448276, 0.        , 0.07427814, ..., 0.13130643, 0.21971769,
        1.        ]])

In [50]:
indices2 = pd.Series(df_2.index)
indices2[:2]

0    Norm of the North: King Sized Adventure
1                 Jandino: Whatever it Takes
Name: title, dtype: object

In [51]:
def recommendations2(title, cosine_sim=cosine_sim2):
    recommended_movies2 = []

    idx = indices2[indices2==title].index[0]

    score_series = pd.Series(cosine_sim2[idx]).sort_values(ascending=False)

    top10_indexes = list(score_series.iloc[1:16].index)

    for i in top10_indexes:
        recommended_movies2.append(list(df_2.index)[i])

    return recommended_movies2

In [52]:
recommendations2('Breaking Bad')

['Have You Ever Fallen in Love, Miss Jiang?',
 'El Reemplazante',
 'Better Call Saul',
 'Unit 42',
 'Pyaar Tune Kya Kiya',
 'The Writer',
 'Age of Rebellion',
 'Jack Taylor',
 'Re:Mind',
 'Get Shorty',
 'Mind Game',
 'Sparta',
 'Ex-Boyfriend',
 'Blood Pact',
 'The Judgement']

In [53]:
recommendations2('Stranger Things')

['Beyond Stranger Things',
 'Star-Crossed',
 'Anjaan: Special Crimes Unit',
 'The OA',
 'Nightflyers',
 'Once Upon a Time',
 'Helix',
 'Jonathan Strange & Mr Norrell',
 'The 4400',
 'The Messengers',
 'Haven',
 'Maharakshak Devi',
 'Scream',
 'Zoo',
 'Prank Encounters']

In [54]:
recommendations2('Like Father')

['Queen',
 'Love Off the Cuff',
 'New Money',
 'Welcome to the Family',
 'Haapus',
 'Mad World',
 'Septiembre, un llanto en silencio',
 'Temporada de Caza',
 'Kodachrome',
 'From Japan to Egypt',
 'The Daughter',
 '10 jours en or',
 'Dear Dad',
 'Walking Out',
 "You're Everything To Me"]

In [54]:
from sklearn.externals import joblib
joblib.dump(cosine_sim2, 'modelo_muvick_entrenado.pkl')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['modelo_muvick_entrenado.pkl']

In [61]:
df_2.to_csv('/content/drive/My Drive/proyecto Netflix/df.csv', index=True)